# Modelagem preditiva
_Machine Learning_

---

## Sumário

1. **Importação de bibliotecas**
2. **Carregamento das bases**
3. **Análise dos dataframes**
4. **Modelagem preditiva**
    - 4.1. Preparação dos dados
    - 4.2. Treinamento dos modelos com todo o histórico de dados
    - 4.3. Treinamento dos modelos com histórico de dados a partir de 01-01-2014
    - 4.4. Treinamento dos modelos com histórico de dados a partir de 01-01-2015
    - 4.5. Comparativo dos resultados
5. **Tunagem dos hiperparâmetros dos melhores algoritmos**
    - 5.1. Tunagem do modelo LightGBM com Optuna
    - 5.2. Tunagem do modelo XGBoost com Optuna
6. **Salvando os modelos**
    - 6.1. Salvando e testando o desempenho dos modelos
    - 6.2. Comparativo dos resultados


<br>

---

<br>

## 1. Importação de bibliotecas

In [ ]:
# Importação de pacotes e definição de parâmetros globais

import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import gc
import time
import optuna
import joblib
import lightgbm as lgb
import catboost as cb

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from optuna.samplers import TPESampler

In [2]:
# Configurações para exibição de dados no Jupyter Notebook

# Configurar opção para exibir todas as linhas do Dataframe
pd.set_option('display.max_rows', None)

# Configurar para exibir o conteúdo completo das colunas
pd.set_option('display.max_colwidth', None)

# Configurar a supressão de mensagens de aviso durante a execução
warnings.filterwarnings('ignore')

# Configurar estilo dos gráficos do seaborn
sns.set_style('whitegrid')

## 2. Carregamento das bases

In [3]:
# Efetuando a limpeza da memória antes do carregamento dos dados

print(f'\nQuantidade de objetos removidos da memória: {gc.collect()}')


Quantidade de objetos removidos da memória: 5


In [4]:
# Criando um dataframe a partir do arquivo train_001.csv

df_train = pd.read_csv('dados/train_001.csv', sep=',')
print('\nDATAFRAME: df_train')
df_train.head()


DATAFRAME: df_train


,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,...,PromoRollingSum_21,PromoRollingSum_28,PromoRollingSum_30,PromoRollingSum_60,PromoRollingSum_90,PromoRollingSum_120,PromoRollingSum_150,PromoRollingSum_180,PromoRollingSum_360,Target
0,1,1,2015-06-30,1,1,0,0,c,a,1270.0,...,9.0,10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,149389.0
1,2,1,2015-06-30,1,1,0,0,a,a,570.0,...,9.0,10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,170586.0
2,3,1,2015-06-30,1,1,0,1,a,a,14130.0,...,9.0,10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,243725.0
3,4,1,2015-06-30,1,1,0,0,c,c,620.0,...,9.0,10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,334601.0
4,5,1,2015-06-30,1,1,0,0,a,a,29910.0,...,9.0,10.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,162182.0


In [5]:
# Criando um dataframe a partir do arquivo validation_001.csv

df_validation = pd.read_csv('dados/validation_001.csv', sep=',')
print('\nDATAFRAME: df_validation')
df_validation.head()


DATAFRAME: df_validation


,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,...,PromoRollingSum_21,PromoRollingSum_28,PromoRollingSum_30,PromoRollingSum_60,PromoRollingSum_90,PromoRollingSum_120,PromoRollingSum_150,PromoRollingSum_180,PromoRollingSum_360,Target
0,1,4,2015-07-31,1,1,0,1,c,a,1270.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,157057.0
1,2,4,2015-07-31,1,1,0,1,a,a,570.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,176872.0
2,3,4,2015-07-31,1,1,0,1,a,a,14130.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,245876.0
3,4,4,2015-07-31,1,1,0,1,c,c,620.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,355383.0
4,5,4,2015-07-31,1,1,0,1,a,a,29910.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,162083.0


## 3. Análise dos dataframes

In [6]:
# Exibindo a quantidade de linhas e colunas dos dataframes

# Criação de um dicionário com os dataframes e seus respectivos nomes
dfs = {
    'df_train': df_train,
    'df_validation': df_validation
}

# Iteração sobre o dicionário para exibir o nome e as dimensões dos dataframes
print(f'\nVOLUMETRIA')
for nome, df in dfs.items():
    print(f'\n{nome}')
    print(f'-'*45)
    print(f'Quantidade de linhas (registros):  {df.shape[0]}')
    print(f'Quantidade de colunas (variáveis): {df.shape[1]}')    


VOLUMETRIA

df_train
---------------------------------------------
Quantidade de linhas (registros):  943795
Quantidade de colunas (variáveis): 38

df_validation
---------------------------------------------
Quantidade de linhas (registros):  34565
Quantidade de colunas (variáveis): 38


In [7]:
# Função para geração de um dataframe de metadados

def gerar_metadados(dataframe):
    '''
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: Dataframe
        DataFrame para o qual os metadados serão gerados.
    :return: DataFrame
        DataFrame contendo os metadados.
    '''
    metadados = pd.DataFrame({
        'Variável': dataframe.columns,
        'Tipo': dataframe.dtypes,
        'Qtde de nulos': dataframe.isnull().sum(),
        '% de nulos': round((dataframe.isnull().sum()/len(dataframe))*100, 2),
        'Cardinalidade': dataframe.nunique(),
    })
    metadados = metadados.sort_values(by='Qtde de nulos', ascending=False)
    metadados = metadados.reset_index(drop=True)
    return metadados

In [8]:
gerar_metadados(df_train)

,Variável,Tipo,Qtde de nulos,% de nulos,Cardinalidade
0,Store,int64,0,0.0,1115
1,PromoRollingSum_21,float64,0,0.0,7
2,DayOfYear,int64,0,0.0,365
3,MonthsSinceTheCompetitionOpened,float64,0,0.0,336
4,YearsSinceTheCompetitionOpened,float64,0,0.0,346
5,PromoRollingSum_3,float64,0,0.0,4
6,PromoRollingSum_5,float64,0,0.0,6
7,PromoRollingSum_7,float64,0,0.0,6
8,PromoRollingSum_14,float64,0,0.0,11
9,PromoRollingSum_28,float64,0,0.0,11


## 4. Modelagem preditiva

### 4.1. Preparação dos dados

In [9]:
# Separando as variáveis preditivas e a variável preditora (alvo)

features = df_train.columns.drop('Target')
target = 'Target'

In [10]:
# Separando as variáveis numéricas e categóricas

numerical_features = df_train[features].select_dtypes(exclude=object).columns
categorical_features = df_train[features].select_dtypes(include=object).columns

In [11]:
# Converter todas as colunas categóricas para string

df_train[categorical_features] = df_train[categorical_features].astype(str)
df_validation[categorical_features] = df_validation[categorical_features].astype(str)

In [12]:
# Separando os dataframes com as variáveis preditivas e a variável preditora

X_train = df_train[features]
y_train = df_train[target]
X_test = df_validation[features]
y_test = df_validation[target]

In [13]:
# Pré-processamento, transformação das features numéricas e categóricas

preprocessor = ColumnTransformer(
	transformers=[
		('num', StandardScaler(), numerical_features),
		('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

In [14]:
# Verificando os shapes

print(f'Shape: X_train: {X_train.shape}, y_train: {y_train.shape}')
print(f'Shape: X_test:  {X_test.shape},  y_test: {y_test.shape}')

Shape: X_train: (943795, 37), y_train: (943795,)
Shape: X_test:  (34565, 37),  y_test: (34565,)


In [15]:
# Verificando transformação de dados

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [16]:
# Verificando os shapes após transformação

print(f'Shape de X_train após transformação: {X_train_transformed.shape}')
print(f'Shape de X_test após transformação:  {X_test_transformed.shape}')

Shape de X_train após transformação: (943795, 958)
Shape de X_test após transformação:  (34565, 958)


### 4.2. Treinamento dos modelos com todo o histórico de dados

In [17]:
# Definindo os modelos a serem testados

models = {
    'LinearRegression': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1),
    'CatBoost': cb.CatBoostRegressor(iterations=100, depth=6, learning_rate=0.1, loss_function='RMSE', verbose=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
}

In [18]:
def models_evaluation(models, X_train, y_train, X_test, y_test, preprocessor):
    '''
    Avalia modelos de aprendizado de máquina, calculando métricas de desempenho 
    no conjunto de treino e teste.

    :param models: dict
        Dicionário contendo os modelos a serem avaliados.
    :param X_train: DataFrame
        Conjunto de dados de treino com as variáveis independentes.
    :param y_train: Series
        Variável dependente para o conjunto de treino.
    :param X_test: DataFrame
        Conjunto de dados de teste com as variáveis independentes.
    :param y_test: Series
        Variável dependente para o conjunto de teste.
    :param preprocessor: ColumnTransformer
        Objeto de pré-processamento que será aplicado aos dados antes do treinamento do modelo.

    :return: list
        Lista contendo os resultados de avaliação de cada modelo, RMSE e MAE para treino 
        e teste, além do tempo de execução.
    '''
    results = []

    for model_name, model in models.items():
        # Cria um pipeline que combina o pré-processamento e o modelo
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

        # Registra o tempo de início
        start_time = time.time()
        
        # Treina o modelo usando o pipeline
        pipeline.fit(X_train, y_train)
        
        # Registra o tempo de término e calcula o tempo de execução
        end_time = time.time()
        elapsed_time = end_time - start_time

        # Previsões e cálculo das métricas no conjunto de treino
        train_predictions = pipeline.predict(X_train)
        train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
        train_mae = mean_absolute_error(y_train, train_predictions)

        # Previsões e cálculo das métricas no conjunto de teste
        test_predictions = pipeline.predict(X_test)
        test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
        test_mae = mean_absolute_error(y_test, test_predictions)

        # Adiciona os resultados à lista
        results.append((
            model_name, train_rmse, test_rmse, train_mae, test_mae, elapsed_time
        ))

    return results

In [19]:
# Avaliando os modelos
results = models_evaluation(models, X_train, y_train, X_test, y_test, preprocessor)

# Criando DataFrame de resultados
results_df = pd.DataFrame(results, columns=[
                            'Modelo', 'RMSE em treino', 'RMSE em teste', 
                            'MAE em treino', 'MAE em teste', 'Tempo decorrido (s)'])

# Exibindo os resultados
results_df

,Modelo,RMSE em treino,RMSE em teste,MAE em treino,MAE em teste,Tempo decorrido (s)
0,LinearRegression,74600.379064,309293.409817,51738.388898,271276.664045,31.595168
1,LightGBM,38586.776903,44161.800849,27936.856770,32487.902792,9.428456
2,CatBoost,56395.651572,62284.344024,39903.499101,44085.404224,15.095588
3,GradientBoosting,65562.258359,70611.741017,45250.375402,48747.665342,334.461325
4,XGBoost,21462.326807,28607.617056,15610.326833,21221.732968,9.710920


### 4.3. Treinamento dos modelos com histórico de dados a partir de 01-01-2014

In [20]:
# Selecionando dados a partir de 01-01-2014
df_train_00 = df_train[df_train['Date'] >= '2014-01-01'].copy()

# Converter todas as colunas categóricas para string
df_train_00[categorical_features] = df_train_00[categorical_features].astype(str)

# Separando os dataframes com as variáveis preditivas e a variável preditora
X_train = df_train_00[features]
y_train = df_train_00[target]

# Pré-processamento, transformação das features numéricas e categóricas
preprocessor = ColumnTransformer(
	transformers=[
		('num', StandardScaler(), numerical_features),
		('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Avaliando os modelos
results = models_evaluation(models, X_train, y_train, X_test, y_test, preprocessor)

# Criando DataFrame de resultados
results_df_00 = pd.DataFrame(results, columns=[
                            'Modelo', 'RMSE em treino', 'RMSE em teste', 
                            'MAE em treino', 'MAE em teste', 'Tempo decorrido (s)'])

# Exibindo os resultados
results_df_00

,Modelo,RMSE em treino,RMSE em teste,MAE em treino,MAE em teste,Tempo decorrido (s)
0,LinearRegression,74409.362859,276823.117601,51872.836322,240122.218335,17.609226
1,LightGBM,37508.232887,42825.956651,27228.816279,31749.433281,4.844280
2,CatBoost,56252.880219,61747.624779,39709.751946,43654.323788,10.245021
3,GradientBoosting,64445.219168,69331.655819,44933.129775,48135.930325,197.969966
4,XGBoost,19679.021450,25890.517653,14269.371007,19121.840148,5.249812


### 4.4. Treinamento dos modelos com histórico de dados a partir de 01-01-2015

In [21]:
# Selecionando dados a partir de 01-01-2015
df_train_01 = df_train[df_train['Date'] >= '2015-01-01'].copy()

# Converter todas as colunas categóricas para string
df_train_01[categorical_features] = df_train_01[categorical_features].astype(str)

# Separando os dataframes com as variáveis preditivas e a variável preditora
X_train = df_train_01[features]
y_train = df_train_01[target]

# Pré-processamento, transformação das features numéricas e categóricas
preprocessor = ColumnTransformer(
	transformers=[
		('num', StandardScaler(), numerical_features),
		('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Avaliando os modelos
results = models_evaluation(models, X_train, y_train, X_test, y_test, preprocessor)

# Criando DataFrame de resultados
results_df_01 = pd.DataFrame(results, columns=[
                            'Modelo', 'RMSE em treino', 'RMSE em teste', 
                            'MAE em treino', 'MAE em teste', 'Tempo decorrido (s)'])

# Exibindo os resultados
results_df_01

,Modelo,RMSE em treino,RMSE em teste,MAE em treino,MAE em teste,Tempo decorrido (s)
0,LinearRegression,73699.366946,86771.077084,51404.782369,60645.653521,2.180020
1,LightGBM,35155.499255,40311.140946,25778.097047,29902.591028,1.834738
2,CatBoost,54854.734524,62294.912910,38846.806357,43603.607236,2.678659
3,GradientBoosting,63451.163306,69028.536458,44110.979695,47900.685039,55.165528
4,XGBoost,18390.785339,24604.677229,13493.532465,18316.927099,1.678125


### 4.5. Comparativo dos resultados

In [22]:
# Utilizando todo o histórico de dados
# Exibindo os resultados
results_df

,Modelo,RMSE em treino,RMSE em teste,MAE em treino,MAE em teste,Tempo decorrido (s)
0,LinearRegression,74600.379064,309293.409817,51738.388898,271276.664045,31.595168
1,LightGBM,38586.776903,44161.800849,27936.856770,32487.902792,9.428456
2,CatBoost,56395.651572,62284.344024,39903.499101,44085.404224,15.095588
3,GradientBoosting,65562.258359,70611.741017,45250.375402,48747.665342,334.461325
4,XGBoost,21462.326807,28607.617056,15610.326833,21221.732968,9.710920


Os valores elevados de **RMSE** e **MAE** podem ser atribuídos à grande variação na variável dependente _'Target'_, que inclui dias com vendas zero e dias com vendas muito elevadas. Esse cenário reflete a realidade de que drogarias podem estar fechadas ou operar em horários reduzidos, o que influencia os dados. O foco, no entanto, deve ser nas diferenças entre as métricas de treino e teste, o que indica a capacidade dos modelos em **generalizar**.

Dentre os modelos analisados, o **LinearRegression** teve o pior desempenho com os maiores valores de erro. O **GradientBoosting** apresentou boas diferenças entre treino e teste, mas os erros ainda foram elevados, além de apresentar o pior tempo de execução. O **CatBoost** se destacou com métricas razoáveis, mas seu tempo de execução foi elevado em relação aos próximos modelos. O **XGBoost** obteve os menores valores de _RMSE_ e _MAE_ em comparação aos demais modelos, porém com uma diferença maior entre treino e teste, indicando possível overfitting. O **LightGBM**, com _RMSE_ e _MAE_ competitivos e o menor tempo de execução, mostrou-se o modelo mais equilibrado, oferecendo a melhor performance geral.

## 5. Tunagem dos hiperparâmetros dos melhores algoritmos

### 5.1. Tunagem do modelo LightGBM com Optuna

In [23]:
def objective_lightgbm(trial):
    '''
    Função objetivo para otimizar os hiperparâmetros do modelo LGBMRegressor usando o Optuna.

    :param trial: optuna.Trial
        Objeto que sugere valores para os hiperparâmetros do modelo durante o processo de otimização.

    :return: float
        O valor do MAE calculado no conjunto de teste para o modelo treinado.
    '''   
    model = lgb.LGBMRegressor(
        # Número de árvores (boosting rounds) no modelo, cada árvore corrige os erros da anterior
        n_estimators = trial.suggest_int('n_estimators', 300, 450),
        # Taxa de aprendizado para ajustar o impacto de cada árvore
        learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
        # Limita a profundidade máxima de cada árvore, controlando o crescimento da árvore e prevenindo overfitting
        max_depth = trial.suggest_int('max_depth', 3, 5),
        # Número máximo de folhas em cada árvore. Controla a complexidade do modelo
        num_leaves = trial.suggest_int('num_leaves', 20, 99),
        # Proporção de amostras usadas para treinar cada árvore, útil para prevenir overfitting
        subsample = trial.suggest_float('subsample', 0.4, 0.7),
        # Proporção de colunas (features) usadas para construir cada árvore, ajuda a reduzir correlação entre árvores e evitar overfitting
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.7, 1.0),
        # Regularização L1 (Lasso), adiciona penalidade proporcional ao valor absoluto dos coeficientes, ajudando a reduzir o overfitting
        reg_alpha = trial.suggest_float('reg_alpha', 1, 10, log=True),
        # Regularização L2 (Ridge), adiciona penalidade proporcional ao quadrado dos coeficientes, ajudando a reduzir o overfitting
        reg_lambda = trial.suggest_float('reg_lambda', 1, 10, log=True),
        # Semente aleatória para garantir reprodutibilidade dos resultados     
        random_state=42,
        # Silenciar a saída de logs durante o treinamento
        verbose=-1,         
    )

    # Cria um pipeline que combina o pré-processamento e o modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    
    # Treina o modelo usando o pipeline
    pipeline.fit(X_train, y_train)
   
    # Previsões e cálculo das métricas no conjunto de teste
    test_predictions = pipeline.predict(X_test)
    test_mae = mean_absolute_error(y_test, test_predictions)
    
    return test_mae

In [24]:
# Cria o estudo Optuna com o sampler TPE
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))

# Otimiza o modelo com a função objetivo
study.optimize(objective_lightgbm, n_trials=100)

[I 2024-11-10 00:00:51,834] A new study created in memory with name: no-name-f7ea192f-07ab-4d5c-921b-104cdd9783fe
[I 2024-11-10 00:00:56,063] Trial 0 finished with value: 34636.81886732525 and parameters: {'n_estimators': 356, 'learning_rate': 0.044635901521768134, 'max_depth': 5, 'num_leaves': 67, 'subsample': 0.44680559213273097, 'colsample_bytree': 0.7467983561008608, 'reg_alpha': 1.1430983876313219, 'reg_lambda': 7.348118405270449}. Best is trial 0 with value: 34636.81886732525.
[I 2024-11-10 00:00:59,011] Trial 1 finished with value: 48634.26465549473 and parameters: {'n_estimators': 390, 'learning_rate': 0.025529516046973785, 'max_depth': 3, 'num_leaves': 97, 'subsample': 0.6497327922401265, 'colsample_bytree': 0.7637017332034828, 'reg_alpha': 1.5199348301309807, 'reg_lambda': 1.5254729458052607}. Best is trial 0 with value: 34636.81886732525.
[I 2024-11-10 00:01:03,229] Trial 2 finished with value: 47874.91299773548 and parameters: {'n_estimators': 345, 'learning_rate': 0.016738

In [25]:
# Acessa o melhor MAE e os melhores parâmetros após a otimização
mae_lightgbm = study.best_value
params_lightgbm = study.best_params

print(f'Melhores hiperparâmetros: {study.best_params}')
print(f'\nMelhor MAE: {study.best_value:.4f}')

Melhores hiperparâmetros: {'n_estimators': 450, 'learning_rate': 0.049822113148944844, 'max_depth': 5, 'num_leaves': 40, 'subsample': 0.51956090741459, 'colsample_bytree': 0.9341156853653367, 'reg_alpha': 6.056451913926463, 'reg_lambda': 8.530066377434268}

Melhor MAE: 29409.3227


### 5.2. Tunagem do modelo XGBoost com Optuna

In [26]:
def objective_xgboost(trial):
    '''
    Função objetivo para otimizar os hiperparâmetros do modelo XGBRegressor usando o Optuna.

    :param trial: optuna.Trial
        Objeto que sugere valores para os hiperparâmetros do modelo durante o processo de otimização.

    :return: float
        O valor do MAE calculado no conjunto de teste para o modelo treinado.
    '''   
    model = XGBRegressor(
        # Número de árvores (boosting rounds) no modelo, cada árvore corrige os erros da anterior
        n_estimators=trial.suggest_int('n_estimators', 300, 450),        
        # Taxa de aprendizado para ajustar o impacto de cada árvore
        learning_rate=trial.suggest_float('learning_rate', 0.005, 0.05, log=True),        
        # Limita a profundidade máxima de cada árvore, controlando o crescimento da árvore e prevenindo overfitting
        max_depth=trial.suggest_int('max_depth', 3, 5),        
        # Número máximo de folhas em cada árvore. Controla a complexidade do modelo
        max_leaves=trial.suggest_int('max_leaves', 20, 99),        
        # Proporção de amostras usadas para treinar cada árvore, útil para prevenir overfitting
        subsample=trial.suggest_float('subsample', 0.4, 0.7),        
        # Proporção de colunas (features) usadas para construir cada árvore, ajuda a reduzir correlação entre árvores e evitar overfitting
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.7, 1.0),        
        # Regularização L1 (Lasso), adiciona penalidade proporcional ao valor absoluto dos coeficientes, ajudando a reduzir o overfitting
        reg_alpha=trial.suggest_float('reg_alpha', 1, 10, log=True),
        # Regularização L2 (Ridge), adiciona penalidade proporcional ao quadrado dos coeficientes, ajudando a reduzir o overfitting
        reg_lambda=trial.suggest_float('reg_lambda', 1, 10, log=True),
        # Semente aleatória para garantir reprodutibilidade dos resultados
        random_state=42,
        # Silenciar a saída de logs durante o treinamento
        verbosity=0,
    )

    # Cria um pipeline que combina o pré-processamento e o modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    
    # Treina o modelo usando o pipeline
    pipeline.fit(X_train, y_train)
   
    # Faz previsões no conjunto de teste e calcula as métricas
    test_predictions = pipeline.predict(X_test)
    test_mae = mean_absolute_error(y_test, test_predictions)
    
    return test_mae

In [27]:
# Cria o estudo Optuna com o sampler TPE
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))

# Otimiza o modelo com a função objetivo
study.optimize(objective_xgboost, n_trials=100)

[I 2024-11-10 00:08:51,535] A new study created in memory with name: no-name-5144df34-71ca-4b81-8f39-d67fb79fa294
[I 2024-11-10 00:08:55,927] Trial 0 finished with value: 35203.41371763931 and parameters: {'n_estimators': 356, 'learning_rate': 0.044635901521768134, 'max_depth': 5, 'max_leaves': 67, 'subsample': 0.44680559213273097, 'colsample_bytree': 0.7467983561008608, 'reg_alpha': 1.1430983876313219, 'reg_lambda': 7.348118405270449}. Best is trial 0 with value: 35203.41371763931.
[I 2024-11-10 00:09:00,072] Trial 1 finished with value: 48485.85009266873 and parameters: {'n_estimators': 390, 'learning_rate': 0.025529516046973785, 'max_depth': 3, 'max_leaves': 97, 'subsample': 0.6497327922401265, 'colsample_bytree': 0.7637017332034828, 'reg_alpha': 1.5199348301309807, 'reg_lambda': 1.5254729458052607}. Best is trial 0 with value: 35203.41371763931.
[I 2024-11-10 00:09:05,645] Trial 2 finished with value: 47823.71964918297 and parameters: {'n_estimators': 345, 'learning_rate': 0.016738

In [28]:
# Acessa o melhor MAE e os melhores parâmetros após a otimização
mae_xgboost = study.best_value
params_xgboost = study.best_params

print(f'Melhores hiperparâmetros: {study.best_params}')
print(f'\nMelhor MAE: {study.best_value:.4f}')

Melhores hiperparâmetros: {'n_estimators': 450, 'learning_rate': 0.04991816070960025, 'max_depth': 5, 'max_leaves': 37, 'subsample': 0.4127560059076609, 'colsample_bytree': 0.9624760048803387, 'reg_alpha': 1.668200985120503, 'reg_lambda': 5.196909694919349}

Melhor MAE: 29898.7066


## 6. Salvando os modelos

### 6.1. Salvando e testando o desempenho dos modelos

In [29]:
def train_and_save_model(best_model, file_name, model_name):
    '''
    Treina um modelo com os melhores hiperparâmetros, salva o pipeline treinado 
    em um arquivo e avalia o desempenho no conjunto de teste.

    :param best_model: Estimator object
        O modelo otimizado com os melhores hiperparâmetros a ser treinado.
    :param file_name: str
        Nome do arquivo para salvar o pipeline treinado (formato .pkl).
    :param model_name: str
        Nome do modelo para identificação nos resultados.

    :return: list
        Lista contendo o nome do modelo, o RMSE e o MAE calculados no conjunto 
        de teste, para avaliação do desempenho.
    '''
    results = []
    
    # Recriando o pipeline com os melhores hiperpârametros
    pipeline_best_model = Pipeline(steps=[('preprocessor', preprocessor),
                                          ('model', best_model)])
    
    # Treinando o pipeline final no conjunto de treinamento
    pipeline_best_model.fit(X_train, y_train)
    
    try:
        # Salvando o pipeline em um arquivo .pkl
        joblib.dump(pipeline_best_model, f'modelos/{file_name}.pkl')
        print(f'Arquivo {file_name}.pkl gerado com sucesso!')
    except Exception as e:
        print(f'Erro ao salvar o pipeline: {e}')
        
    # Previsões e cálculo das métricas no conjunto de treino
    train_predictions = pipeline_best_model.predict(X_train)
    train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
    train_mae = mean_absolute_error(y_train, train_predictions)
    
    # Previsões e cálculo das métricas no conjunto de teste
    test_predictions = pipeline_best_model.predict(X_test)
    test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
    test_mae = mean_absolute_error(y_test, test_predictions)
    
    # Adiciona previsões em teste a lista
    results.append((
        model_name, train_rmse, test_rmse, train_mae, test_mae
    ))
    
    return results

In [30]:
# Parâmetros otimizados para o LightGBM
best_params = params_lightgbm

# Cria o modelo com os melhores hiperparâmetros
best_model = lgb.LGBMRegressor(**best_params, verbose=-1)

# Treina o modelo e salva o pipeline treinado em um arquivo .pkl
results_lightgbm = train_and_save_model(best_model, 'pipeline_best_model_lightgbm', 'LightGBM')

Arquivo pipeline_best_model_lightgbm.pkl gerado com sucesso!


In [31]:
# Parâmetros otimizados para o XGBoost
best_params = params_xgboost

# Cria o modelo com os melhores hiperparâmetros
best_model = XGBRegressor(**best_params, verbosity=0)

# Treina o modelo e salva o pipeline treinado em um arquivo .pkl
results_xgboost = train_and_save_model(best_model, 'pipeline_best_model_xgboost', 'XGBoost')

Arquivo pipeline_best_model_xgboost.pkl gerado com sucesso!


### 6.2. Comparativo dos resultados

In [32]:
# Definindo as colunas para o DataFrame
columns = ['Modelo', 'RMSE em treino', 'RMSE em teste', 'MAE em treino', 'MAE em teste']

# Criando DataFrames com os resultados dos modelos LightGBM e XGBoost
df_results_lightgbm = pd.DataFrame(results_lightgbm, columns=columns)
df_results_xgboost = pd.DataFrame(results_xgboost, columns=columns)

# Concatenando os resultados dos dois modelos em um único DataFrame
df_results = pd.concat([df_results_lightgbm, df_results_xgboost], ignore_index=True)

In [33]:
print('\nResultado dos modelos SEM a tunagem de hiperparâmetros')
results_df.iloc[[1, 4], 0:5]


Resultado dos modelos SEM a tunagem de hiperparâmetros


,Modelo,RMSE em treino,RMSE em teste,MAE em treino,MAE em teste
1,LightGBM,38586.776903,44161.800849,27936.856770,32487.902792
4,XGBoost,21462.326807,28607.617056,15610.326833,21221.732968


In [34]:
print('\nResultado dos modelos COM a tunagem de hiperparâmetros')
df_results


Resultado dos modelos COM a tunagem de hiperparâmetros


,Modelo,RMSE em treino,RMSE em teste,MAE em treino,MAE em teste
0,LightGBM,36887.215123,41674.877522,26525.398543,30222.956458
1,XGBoost,35901.282846,40779.170533,26049.486890,29729.058447


A tunagem de hiperparâmetros trouxe uma melhoria na redução da diferença entre os valores das métricas **RMSE** e **MAE** nos conjuntos de dados de **treino** e **teste**, o que indica que os ajustes de hiperparâmetros ajudaram os modelos a capturar melhor os padrões dos dados. No entanto, ao analisar os erros, observou-se um comportamento distinto entre os modelos **LightGBM** e **XGBoost**.

O modelo **XGBoost** apresentou um aumento nas métricas de erro **RMSE** e **MAE**, o que sugere que os ajustes de hiperparâmetros aumentaram a complexidade do modelo. Apesar disso, o **XGBoost** ainda demonstrou uma boa redução nas diferenças entre as métricas dos dados de treino e teste. Isso indica que mais ajustes ou outras abordagens seriam necessárias para melhorar os valores das métricas e reduzir as diferenças entre treino e teste.

Por outro lado, o modelo **LightGBM** apresentou uma leve melhoria nas métricas de erro **RMSE** e **MAE** após a tunagem, indicando que o modelo agora está prevendo com erros menores e com uma melhor capacidade de **generalização**. Houve uma redução tanto nos valores das métricas quanto nas diferenças entre os resultados de treino e teste. O **LightGBM** continua se destacando como o modelo mais equilibrado, com redução das métricas de **RMSE** e **MAE**, aliado à melhor capacidade de generalização.